In [48]:
import plotly.graph_objects as go
import numpy as np
from typing import Optional
import os.path as osp
import os

def to_homogeneous(points):
    pad = np.ones((points.shape[:-1]+(1,)), dtype=points.dtype)
    return np.concatenate([points, pad], axis=-1)
def init_figure(height: int = 800) -> go.Figure:
    """Initialize a 3D figure."""
    fig = go.Figure()
    axes = dict(
        visible=False,
        showbackground=False,
        showgrid=False,
        showline=False,
        showticklabels=True,
        autorange=True,
    )
    fig.update_layout(
        template="plotly_white",
        height=height,
        scene_camera=dict(
            eye=dict(x=0., y=-.1, z=-2),
            up=dict(x=0, y=-1., z=0),
            projection=dict(type="perspective")),
        scene=dict(
            xaxis=axes,
            yaxis=axes,
            zaxis=axes,
            aspectmode='data',
            dragmode='orbit',
        ),
        margin=dict(l=0, r=0, b=0, t=0, pad=0),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.1
        ),
    )
    return fig

def plot_camera(
        fig: go.Figure,
        R: np.ndarray,
        t: np.ndarray,
        K: np.ndarray,
        color: str = 'rgb(0, 0, 255)',
        name: Optional[str] = None,
        legendgroup: Optional[str] = None,
        size: float = 1.0):
    """Plot a camera frustum from pose and intrinsic matrix."""
    W, H = K[0, 2]*2, K[1, 2]*2
    corners = np.array([[0, 0], [W, 0], [W, H], [0, H], [0, 0]])
    if size is not None:
        image_extent = max(size * W / 1024.0, size * H / 1024.0)
        world_extent = max(W, H) / (K[0, 0] + K[1, 1]) / 0.5
        scale = 0.5 * image_extent / world_extent
    else:
        scale = 1.0
    corners = to_homogeneous(corners) @ np.linalg.inv(K).T
    corners = (corners / 2 * scale) @ R.T + t

    x, y, z = corners.T
    rect = go.Scatter3d(
        x=x, y=y, z=z, line=dict(color=color), legendgroup=legendgroup,
        name=name, marker=dict(size=0.0001), showlegend=False)
    fig.add_trace(rect)

    x, y, z = np.concatenate(([t], corners)).T
    i = [0, 0, 0, 0]
    j = [1, 2, 3, 4]
    k = [2, 3, 4, 1]

    pyramid = go.Mesh3d(
        x=x, y=y, z=z, color=color, i=i, j=j, k=k,
        legendgroup=legendgroup, name=name, showlegend=False)
    fig.add_trace(pyramid)
    triangles = np.vstack((i, j, k)).T
    vertices = np.concatenate(([t], corners))
    tri_points = np.array([
        vertices[i] for i in triangles.reshape(-1)
    ])

    x, y, z = tri_points.T
    pyramid = go.Scatter3d(
        x=x, y=y, z=z, mode='lines', legendgroup=legendgroup,
        name=name, line=dict(color=color, width=1), showlegend=False)
    fig.add_trace(pyramid)

In [9]:
fig = init_figure()

K = np.array([[512,0,256],
              [0,512,256],
              [0,0,1]])
plot_camera(fig, np.eye(3), np.zeros(3), K)

In [14]:
data_root = '../data/abc/00075213'
camdata = np.load(osp.join(data_root,'cameras.npz'))

In [15]:
intrinsics = camdata['intrinsics']

In [16]:
extrinsics = camdata['extrinsics']

In [17]:
for K, Rt in zip(intrinsics,extrinsics):
    plot_camera(fig,Rt[:3,:3],Rt[:3,3],K)

In [18]:
fig.show()

In [19]:
lines3d = np.load('../exps/dtu-rend-d/24/2023_02_23_10_37_49/wireframes/2000-wfi_checked.npz')

In [21]:
lines3d = lines3d['lines3d']

In [49]:
fig = init_figure()

for line in lines3d:
    x,y,z = line.T
    scatter = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='lines',
        line=dict(color='blue', width=5),
        connectgaps=False,
        showlegend=False
    )

    fig.add_trace(scatter)
    
    scatter = go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(color='red', size=2),
        connectgaps=False,
        showlegend=False
    )
    fig.add_trace(scatter)

fig.show()

In [35]:
x

[0.02632199414074421,
 0.06307262927293777,
 0.02632199414074421,
 0.02921026386320591,
 -0.47193217277526855,
 -0.3886052966117859,
 -0.47193217277526855,
 -0.5239108204841614,
 -0.1694418042898178,
 -0.21019881963729858,
 -0.1694418042898178,
 -0.1662881225347519,
 0.08031637966632843,
 0.1313929408788681,
 0.08031637966632843,
 0.07403459399938583,
 0.004639006685465574,
 -0.07720909267663956,
 0.004639006685465574,
 0.08638760447502136,
 0.004639006685465574,
 -0.003715184284374118,
 0.004639006685465574,
 -0.07417323440313339,
 0.004639006685465574,
 0.06211429461836815,
 -0.0817454382777214,
 -0.12015198171138763,
 -0.0817454382777214,
 -0.0750182569026947,
 -0.3962773084640503,
 -0.4200851023197174,
 -0.3962773084640503,
 -0.37751781940460205,
 -0.5708062052726746,
 -0.6038088202476501,
 -0.5708062052726746,
 -0.5207988023757935,
 -0.3929900527000427,
 -0.38550296425819397,
 -0.3929900527000427,
 -0.38265660405158997,
 -0.3929900527000427,
 -0.4208630919456482,
 -0.2639236748218